In [1]:
import numpy as np
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from google.colab import files

# Upload the zip file
uploaded = files.upload()


Saving homer_bart.zip to homer_bart.zip


In [5]:
# Extract the uploaded zip file
with zipfile.ZipFile('homer_bart.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/homer_bart')

# Directories for the images
base_dir = '/content/homer_bart'
bart_dir = os.path.join(base_dir, 'Bart')
homer_dir = os.path.join(base_dir, 'Homer')

# Create train and test directories
train_bart_dir = os.path.join(base_dir, 'train/Bart')
train_homer_dir = os.path.join(base_dir, 'train/Homer')
test_bart_dir = os.path.join(base_dir, 'test/Bart')
test_homer_dir = os.path.join(base_dir, 'test/Homer')

os.makedirs(train_bart_dir, exist_ok=True)
os.makedirs(train_homer_dir, exist_ok=True)
os.makedirs(test_bart_dir, exist_ok=True)
os.makedirs(test_homer_dir, exist_ok=True)

# Function to split data into train and test
import shutil
import random

def split_data(source, train, test, split_size):
    all_files = os.listdir(source)
    random.shuffle(all_files)
    train_size = int(len(all_files) * split_size)
    train_files = all_files[:train_size]
    test_files = all_files[train_size:]

    for file in train_files:
        shutil.copy(os.path.join(source, file), os.path.join(train, file))
    for file in test_files:
        shutil.copy(os.path.join(source, file), os.path.join(test, file))

# Split Bart and Homer images
split_data(bart_dir, train_bart_dir, test_bart_dir, 0.9)
split_data(homer_dir, train_homer_dir, test_homer_dir, 0.9)


In [16]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import callbacks

# Data augmentation and data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=(128, 128),  # Increased image size
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=(128, 128),  # Increased image size
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained MobileNetV2 model + higher level layers
base_model = MobileNetV2(input_shape=(128, 128, 3),
                         include_top=False,
                         weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator
)


Found 268 images belonging to 2 classes.
Found 53 images belonging to 2 classes.
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
9/9 [==============================] - 10s 628ms/step - loss: 0.8880 - accuracy: 0.5149 - val_loss: 0.7381 - val_accuracy: 0.4151
Epoch 2/10
9/9 [==============================] - 4s 448ms/step - loss: 0.8545 - accuracy: 0.5522 - val_loss: 0.6654 - val_accuracy: 0.5849
Epoch 3/10
9/9 [==============================] - 6s 708ms/step - loss: 0.7254 - accuracy: 0.6082 - val_loss: 0.6356 - val_accuracy: 0.6226
Epoch 4/10
9/9 [==============================] - 4s 461ms/step - loss: 0.6434 - accuracy: 0.6866 - val_loss: 0.5894 - val_accuracy: 0.6981
Epoch 5/10
9/9 [==============================] - 4s 502ms/step - loss: 0.5609 - accuracy: 0.7313 - val_loss: 0.5386 - val_accuracy: 0.7170
Epoch 6/10
9/9 [==============================] - 6s 656ms/step - loss: 0.5581 - accuracy: 0.7239 - val_loss: 0.5002 - val_accuracy: 0.8113
Epoch 7/10
9/9 

In [18]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc * 100)


2/2 [==============================] - 1s 192ms/step - loss: 0.2713 - accuracy: 0.9057
Test Accuracy: 90.56603908538818
